In [1]:
%pip install -qU langchain_ollama

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.2")

llm.invoke("The first man on the moon was ...")



/Users/kjevaji/Code/jupyter/jupyter_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


'...Neil Armstrong. He stepped onto the lunar surface on July 20, 1969 as part of the Apollo 11 mission.'

In [3]:
for chunk in llm.stream("The first man on the moon was ..."):
    print(chunk, end="|", flush=True)

...|Neil| Armstrong|.| He| stepped| onto| the| lunar| surface| on| July| |20|,| |196|9| as| part| of| the| Apollo| |11| mission|.||

In [4]:
from langchain_ollama import ChatOllama

chat_model = ChatOllama(model="llama3.2")

chat_model.invoke("Who was the first man on the moon?")

AIMessage(content='The first man to walk on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon\'s surface on July 20, 1969, during the Apollo 11 mission. His famous words upon setting foot on the Moon were: "That\'s one small step for man, one giant leap for mankind."', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-11-10T06:06:40.100411Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 945167667, 'load_duration': 27752000, 'prompt_eval_count': 34, 'prompt_eval_duration': 162000000, 'eval_count': 69, 'eval_duration': 754000000}, id='run-173be5d0-52f0-4a01-b47f-a54f5ca5448c-0', usage_metadata={'input_tokens': 34, 'output_tokens': 69, 'total_tokens': 103})

In [5]:
# Document loading, retrieval methods and text splitting
%pip install -qU langchain langchain_community

# Local vector store via Chroma
%pip install -qU langchain_chroma

# Local inference and embeddings via Ollama
%pip install -qU langchain_ollama

# Web Loader
%pip install -qU beautifulsoup4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

In [8]:
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [9]:
docs[0]

Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}, page_content='Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.')

In [10]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.2",
)

In [11]:
response_message = model.invoke(
    "Simulate a rap battle between Stephen Colbert and John Oliver"
)

print(response_message.content)

(The scene is set in a dark, crowded nightclub. The audience is on the edge of their seats as two figures emerge from the shadows. On one side, we have Stephen Colbert, dressed in his signature suit and tie, with a confident smirk on his face. Across from him, John Oliver stands tall, his glasses perched atop his nose, and a hint of mischief in his eyes.)

**Round 1: Stephen Colbert**

Yo, I'm the king of late-night fame,
Colbert Report's my claim to fame.
I brought satire to the mainstream game,
While you were still trying to get your name.

My wit is sharp, my humor's tight,
I made fun of politicians with all my might.
From Bush to Trump, I took aim,
Exposing the truth, no matter the claim.

**Round 1: John Oliver**

Hold up, hold up, let me interrupt,
You think you're funny, but your jokes are incorrect.
My Last Week Tonight's where the real talk is done,
I tackle topics that make America numb.

From voter ID to coal mining fees,
I dive deep into issues that affect thee.
No fluff, n

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | model | StrOutputParser()

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

chain.invoke(docs)

'The main themes in these retrieved docs are:\n\n1. Task decomposition: The ability to break down complex tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\n2. Planning: A crucial component that involves identifying and breaking down large tasks into smaller subgoals, and then planning ahead to achieve them.\n3. Expert models: The use of expert models to execute on specific tasks and log results, providing a way for the agent to learn from its mistakes and refine its approach over time.\n4. Self-reflection and refinement: The ability of the agent to reflect on past actions, identify areas for improvement, and refine its approach to achieve better results.\n\nThese themes highlight the key aspects of an LLM-powered autonomous agent system, including planning, task decomposition, expert models, and self-refinement.'

In [13]:
from langchain_core.runnables import RunnablePassthrough

RAG_TEMPLATE = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | model
    | StrOutputParser()
)

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

# Run
chain.invoke({"context": docs, "question": question})

'There are three approaches to task decomposition: (1) using a Large Language Model (LLM) with simple prompting, such as "Steps for XYZ." or "Subgoals for achieving XYZ.", (2) using task-specific instructions like writing a story outline, and (3) with human inputs. These methods enable efficient breakdown of complex tasks into smaller subgoals.'

In [14]:
retriever = vectorstore.as_retriever()

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

In [15]:
question = "What are the approaches to Task Decomposition?"

qa_chain.invoke(question)

'There are three approaches to task decomposition: (1) using simple prompting with a Large Language Model (LLM), such as "Steps for XYZ." or "What are the subgoals for achieving XYZ?", (2) using task-specific instructions, and (3) with human inputs. These approaches enable an autonomous agent system to break down large tasks into smaller, manageable subgoals.'